In [1]:
import os
import json
import pandas as pd

test_root = "data/test"

data_records = []
for kind in os.listdir(test_root):
    path = os.path.join(test_root, kind)
    for f in os.listdir(path):
        if not f.endswith('.json'):
            print("warning: has non-json file")
            continue

        with open(os.path.join(path, f), "r") as j:
            data = json.load(j)
            data["json_number"] = int(f.split('.')[0]) # json_number means 
                                                       # if this data comes from 1.json
                                                       # then it is 1
            data_records.append(data)

df = pd.DataFrame(data_records)
df['id'] = df.index

In [2]:
level = 3
df3 = df[df['level'] == f"Level {level}"]
df3.head()

,problem,level,type,solution,json_number,id
0,How many vertical asymptotes does the graph of...,Level 3,Algebra,The denominator of the rational function facto...,1,0
14,Kite $ABCD$ (a quadrilateral with two pairs of...,Level 3,Algebra,"As the problem suggests, we need to compute th...",1026,14
17,"If $A$, $B$ and $C$ are positive integers such...",Level 3,Algebra,Multiplying the numerator and denominator of t...,1034,17
18,Ten treeks weigh as much as three squigs and o...,Level 3,Algebra,"Let $t,s,g$ be the weight of one treek, the we...",1035,18
23,"At constant temperature, the pressure of a sam...",Level 3,Algebra,Since the pressure $p$ of the hydrogen and the...,1049,23


In [3]:
from datasets import MATH
df.loc[:, 'actual'] = df['solution'].apply(MATH.extract_answer)
df_int = df.dropna(subset=['actual'])
df_int

,problem,level,type,solution,json_number,id,actual
0,How many vertical asymptotes does the graph of...,Level 3,Algebra,The denominator of the rational function facto...,1,0,2.0
1,What is the positive difference between $120\%...,Level 1,Algebra,One hundred twenty percent of 30 is $120\cdot3...,10,1,10.0
4,"If $2^8=4^x$, what is the value of $x$?",Level 1,Algebra,Rewrite $4$ as $2^2$ to find $4^x=2^{2x}$. Si...,1004,4,4.0
5,What is the 100th term of the arithmetic seque...,Level 2,Algebra,"The common difference is $10 - 6 = 4$, so the ...",1009,5,402.0
7,Mr. Madoff invests 1000 dollars in a fund that...,Level 4,Algebra,Let $r$ be the annual interest rate. Then aft...,1014,7,7.0
...,...,...,...,...,...,...,...
4987,Given $\|\mathbf{v}\| = 5$ and $\|\mathbf{w}\|...,Level 3,Precalculus,Note that\n\begin{align*}\n\operatorname{proj}...,98,4987,5.0
4989,If $0^\circ < x < 180^\circ$ and $\cos x + \si...,Level 5,Precalculus,"From the given equation, $\cos x = \frac{1}{2}...",984,4989,14.0
4990,"Let $x_1,$ $x_2,$ $x_3,$ $y_1,$ $y_2,$ and $y_...",Level 5,Precalculus,"In general,\n\[\frac{1}{2} \begin{vmatrix} x_1...",986,4990,144.0
4992,Compute\n\[\frac{1}{\cos^2 10^\circ} + \frac{1...,Level 4,Precalculus,We can write\n\begin{align*}\n\frac{1}{\cos^2 ...,989,4992,12.0


In [7]:
group_sample_size = 2
df_int_gp = df_int.groupby(['level', 'type'])
min_group_size = df_int_gp.apply(lambda group: len(group.index)).min()
if group_sample_size > min_group_size:
    raise Exception("group sample size is larger than the size of a group")
df1 = df_int_gp.apply(lambda group: group.sample(group_sample_size, replace=False))

In [16]:
df2 = df1.apply(lambda group: pd.concat([group, df1['id'] * 10], axis=1))

ValueError: If using all scalar values, you must pass an index

In [9]:
from models.gpt3 import GPT3

my_model = GPT3()

pred = df1['problem'].apply(my_model._predict)

In [14]:
pred.to_csv("gpt3_predictions.csv")

In [35]:
pred.apply(MATH.extract_answer)

level    type                        
Level 1  Algebra                 1008       4.0
                                 497       27.0
         Counting & Probability  1552       3.0
                                 1418       4.0
         Geometry                2008      16.0
                                          ...  
Level 5  Number Theory           3167       1.0
         Prealgebra              3838      36.0
                                 4238    1999.0
         Precalculus             4642       1.0
                                 4880       1.0
Name: problem, Length: 70, dtype: float64

In [37]:
pd.concat([pred.apply(MATH.extract_answer), df1['actual']], axis=1)

# todo: review how you did pandas,
# read https://pandas.pydata.org/docs/user_guide/groupby.html

problem   actual
level   type                                         
Level 1 Algebra                1008      4.0      1.0
                               497      27.0     27.0
        Counting & Probability 1552      3.0      3.0
                               1418      4.0      3.0
        Geometry               2008     16.0     24.0
...                                      ...      ...
Level 5 Number Theory          3167      1.0     24.0
        Prealgebra             3838     36.0     36.0
                               4238   1999.0  10090.0
        Precalculus            4642      1.0     12.0
                               4880      1.0    159.0

[70 rows x 2 columns]

In [7]:
from datasets import MATH
MATH.extract_answer(pred.loc[2803, 'output_lengthy'])

answer in boxed not found


In [4]:
merged_df = pd.merge(df3, pred[['id', 'output']], on='id')
merged_df

,problem,level,type,solution,json_number,id,output
0,"Let $a,$ $b,$ $c$ be positive real numbers. F...",Level 3,Intermediate Algebra,"By QM-AM,\n\[\sqrt{\frac{x^2 + y^2 + z^2}{3}} ...",437,2803,None
1,Given $\mathbf{a} = \begin{pmatrix} 2 \\ 1 \\ ...,Level 3,Precalculus,We have that\n\begin{align*}\n(\mathbf{a} \tim...,1004,4459,None


In [7]:
pip install git+https://github.com/hendrycks/math.git

  Cloning https://github.com/hendrycks/math.git to c:\users\a0306\appdata\local\temp\pip-req-build-ky0t90fv
  Resolved https://github.com/hendrycks/math.git to commit 357963a7f5501a6c1708cf3f3fb0cdf525642761
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for math-equivalence: filename=math_equivalence-0.0.0-py3-none-any.whl size=3509 sha256=072d83a6d524bb41728c6a4a3b239173b2f0894e646470e7778cc2685ae3150a
  Stored in directory: C:\Users\a0306\AppData\Local\Temp\pip-ephem-wheel-cache-yz7idzj1\wheels\b7\16\f0\4a69d4d9b720086e22842cbd2d896b66298e6424b8f289f37c
Successfully built math-equivalence
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/hendrycks/math.git 'C:\Users\a0306\AppData\Local\Temp\pip-req-build-ky0t90fv'


In [2]:
import math_equivalence


In [8]:
y_true = extract_answer(merged_df.loc[0, 'solution'])
y_pred = extract_answer(merged_df.loc[0, 'output'])
print(y_pred, y_true)

47 49


In [10]:
print(merged_df.loc[0, 'problem'])
print("-----------------")
print(merged_df.loc[0, 'output'])

What is the value of $x$ which satisfies $\sqrt[3]{x\sqrt{x}}=7$?
We start by cubing both sides to get rid of the cube root:
$$\sqrt[3]{x\sqrt{x}}=7$$
$$(\sqrt[3]{x\sqrt{x}})^3=7^3$$
$$x\sqrt{x}=343$$
Now, we can square both sides to eliminate the square root:
$$x^2\cdot x=343^2$$
$$x^3=117649$$
Taking the cube root of both sides, we find that $x= \boxed{47}$.


In [12]:
merged_df.loc[0, 'solution']

'Each of the five marked angles measures $360/5=72$ degrees, so $\\boxed{72}$ degrees is the minimum angle through which the pentagon may be rotated so that it coincides with its original position.\n\n[asy]\nsize(150);\ndefaultpen(linewidth(0.7));\nint i;\nfor(i=0;i<=4;++i)\n\n{\ndraw(origin--dir(18+72*i)--dir(18+72*(i+1)));\ndraw(anglemark(dir(18+72*i),origin,dir(18+72*(i+1)),3+fmod(i,3)));\n}\n[/asy]'

Testing contest train data

In [7]:
from models.gpt3 import GPT3

system_prompt = "You are an assistant that is very good at mathematics. Given a mathematics problem, determine the answer, which is always an integer. Put your answer in the box \\boxed{}"

my_model = GPT3(system_prompt=system_prompt)



In [5]:
contest_df = pd.read_csv("train.csv")
contest_df

,id,problem,answer
0,229ee8,"Let $k, l > 0$ be parameters. The parabola $y ...",52
1,246d26,Each of the three-digits numbers $111$ to $999...,250
2,2fc4ad,Let the `sparkle' operation on positive intege...,702
3,430b63,What is the minimum value of $5x^2+5y^2-8xy$ w...,800
4,5277ed,There exists a unique increasing geometric seq...,211
5,739bc9,For how many positive integers $m$ does the eq...,199
6,82e2a0,Suppose that we roll four 6-sided fair dice wi...,185
7,8ee6f3,"The points $\left(x, y\right)$ satisfying $((\...",320
8,bedda4,Let $ABCD$ be a unit square. Let $P$ be the po...,480
9,d7e9c9,A function $f: \mathbb N \to \mathbb N$ satisf...,199


In [8]:
pred = my_model.predict(contest_df[['id', 'problem']]) # 24 seconds

In [9]:
pred

,id,problem,output_lengthy,output
0,229ee8,"Let $k, l > 0$ be parameters. The parabola $y ...",Let's find the $x$-coordinates of points $A$ a...,45.0
1,246d26,Each of the three-digits numbers $111$ to $999...,$\boxed{26}$.,26.0
2,2fc4ad,Let the `sparkle' operation on positive intege...,$\boxed{8}$.,8.0
3,430b63,What is the minimum value of $5x^2+5y^2-8xy$ w...,The minimum value of $5x^2+5y^2-8xy$ is $\boxe...,3200.0
4,5277ed,There exists a unique increasing geometric seq...,"The geometric sequence is $10, 20, 40, 80, 160...",310.0
5,739bc9,For how many positive integers $m$ does the eq...,$\boxed{5}$.,5.0
6,82e2a0,Suppose that we roll four 6-sided fair dice wi...,The total number of outcomes when rolling four...,329.0
7,8ee6f3,"The points $\left(x, y\right)$ satisfying $((\...",The given equation $(\vert x + y \vert - 10)^2...,200.0
8,bedda4,Let $ABCD$ be a unit square. Let $P$ be the po...,The largest region is the triangle formed by t...,NaN
9,d7e9c9,A function $f: \mathbb N \to \mathbb N$ satisf...,We can deduce that $f(n)$ is an increasing fun...,201.0
